# 데이터 불러오기 및 전처리

In [1]:
# 라이브러리
import pandas as pd
import numpy as np
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score,f1_score

In [13]:
df = pd.read_excel('고속도로 통합 데이터(전처리 후)_R2.xlsx',header=0)
# df.head()

#------------------------------------------------------------------------
# 전처리
# 1. 사고 유형 - 단독 사고인 데이터는 [ '피해운전자 성별', '피해운전자 연령,    '피해운전자 상해정도,'피해운전자 차종'] 컬럼의 데이터가 비어 있다.
#    '피해운전자 연령''가해운전자 연령' 두 컬럼을 삭제하고 나머지 컬럼에 대해서는 '기타'로 처리
df = df.drop(['피해운전자 연령', '가해운전자 연령'], axis=1)

df[['피해운전자 성별', '피해운전자 상해정도', '피해운전자 차종']] = df[['피해운전자 성별', '피해운전자 상해정도', '피해운전자 차종']].fillna('기타')

# 사고유형 - 차대차/차대사람
# df = df[df['사고유형']!= '차량단독']

# 사고유형 - 차량단독
# df = df[df['사고유형']== '차량단독']
# df = df.drop(columns=['피해운전자 성별', '피해운전자 상해정도', '피해운전자 차종'])

# 2. 시군구 삭제
df = df.drop('시군구',axis=1)

# 3. '시간' 컬럼을 주간/야간으로 변환
df['시간대'] = df['시간'].apply(lambda x: '주간' if 6 <= x <= 18 else '야간')

# 3. '시간'컴럼의 데이터타입을 object로 변경
df['시간'] = df['시간'].astype('object')

# 4. 사고 심각도(정규성만족을 위해 '사고 경도' 추가)
df['사고 심각도'] = df['사망자수'] * 3 + df['중상자수'] * 2 + df['경상자수']
# df['사고 경도'] = max(df['사고 심각도']) - df['사고 심각도']

# 5. ARI (Accident Risk Index) 계산
df['ARI'] = np.sqrt((df['사망자수'] + df['중상자수'])**2 + 1**2)

df['위험도'] = ''
cond1=1.1
cond2=2
df.loc[df['ARI']<cond1,'위험도'] = '하'
df.loc[(df['ARI']>=cond1)&(df['ARI']<cond2),'위험도'] = '중'
df.loc[df['ARI']>=cond2,'위험도'] = '상'

# df['위험도'].value_counts()



display(df.head())
print(df['위험도'].value_counts())
# df.info()

,시간,계절,요일,주말여부,지역,사고내용,사망자수,중상자수,경상자수,부상신고자수,...,가해운전자 차종,가해운전자 성별,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 상해정도,시간대,사고 심각도,ARI,위험도
0,10,여름,월,주중,경기도,경상사고,0,0,3,5,...,승용,남,상해없음,화물,남,부상신고,주간,3,1.000000,하
1,0,가을,화,주중,경기도,경상사고,0,0,1,5,...,승용,남,상해없음,화물,남,경상,야간,1,1.000000,하
2,17,가을,토,주말,경상북도,경상사고,0,0,1,5,...,승용,여,부상신고,화물,남,상해없음,주간,1,1.000000,하
3,8,봄,화,주중,경기도,부상신고사고,0,0,0,4,...,화물,남,상해없음,화물,남,부상신고,주간,0,1.000000,하
4,11,봄,화,주중,경상남도,중상사고,0,2,1,3,...,화물,남,중상,화물,남,경상,주간,5,2.236068,상


위험도
하    4545
중    1312
상     544
Name: count, dtype: int64


In [24]:
import pandas as pd

# 예시 데이터프레임 (df)가 있다고 가정합니다.
# 위험도가 '중' 또는 '상'인 데이터를 필터링합니다.
df_filtered = df[df['위험도'].isin(['중', '상'])]

# 결과를 저장할 딕셔너리
result = {}

# 각 독립변수에 대해 반복합니다.
independent_vars = ['시간', '계절', '요일', '주말여부', '지역', '사고내용', '사고유형', '법규위반',
                    '노면상태', '기상상태', '도로형태', '가해운전자 차종', '가해운전자 성별',
                    '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 상해정도', '시간대']

for var in independent_vars:
    # 각 독립변수의 전체 데이터 수를 계산합니다.
    total_counts = df[var].value_counts()

    # '중' 또는 '상'인 데이터의 개수를 구합니다.
    filtered_counts = df_filtered[var].value_counts()

    # '중' + '상' 위험도의 비율을 계산합니다.
    proportions = (filtered_counts / len(df)).fillna(0)

    # 가장 비율이 높은 종류를 찾습니다.
    max_category = proportions.idxmax()
    max_proportion = proportions.max()

    # 다른 종류들의 평균 비율을 계산합니다.
    other_mean_proportion = proportions.drop(max_category).mean()

    # 가장 높은 비율과 다른 종류들의 평균 비율 대비 얼마나 높은지 계산합니다.
    relative_increase = (max_proportion - other_mean_proportion)

    # 결과 저장
    result[var] = {
        '최고비율종류': max_category,
        '비율': max_proportion,
        '다른 종류 평균 비율': other_mean_proportion,
        '비율차이(%)': relative_increase * 100
    }

# 결과 출력
for var, info in result.items():
    print(f"{var}에서 '중' 또는 '상' 비율이 가장 높은 종류: {info['최고비율종류']}")
    print(f"비율: {info['비율']:.2%}, 다른 종류 평균 비율: {info['다른 종류 평균 비율']:.2%}, 비율 차이: {info['비율차이(%)']:.2f}%\n")


시간에서 '중' 또는 '상' 비율이 가장 높은 종류: 14
비율: 1.83%, 다른 종류 평균 비율: 1.18%, 비율 차이: 0.65%

계절에서 '중' 또는 '상' 비율이 가장 높은 종류: 가을
비율: 11.97%, 다른 종류 평균 비율: 5.68%, 비율 차이: 6.29%

요일에서 '중' 또는 '상' 비율이 가장 높은 종류: 금
비율: 4.59%, 다른 종류 평균 비율: 4.07%, 비율 차이: 0.53%

주말여부에서 '중' 또는 '상' 비율이 가장 높은 종류: 주중
비율: 21.04%, 다른 종류 평균 비율: 7.95%, 비율 차이: 13.09%

지역에서 '중' 또는 '상' 비율이 가장 높은 종류: 경기도
비율: 9.50%, 다른 종류 평균 비율: 1.39%, 비율 차이: 8.11%

사고내용에서 '중' 또는 '상' 비율이 가장 높은 종류: 중상사고
비율: 24.32%, 다른 종류 평균 비율: 4.67%, 비율 차이: 19.65%

사고유형에서 '중' 또는 '상' 비율이 가장 높은 종류: 차대차
비율: 26.37%, 다른 종류 평균 비율: 1.31%, 비율 차이: 25.06%

법규위반에서 '중' 또는 '상' 비율이 가장 높은 종류: 안전운전불이행
비율: 21.15%, 다른 종류 평균 비율: 1.31%, 비율 차이: 19.85%

노면상태에서 '중' 또는 '상' 비율이 가장 높은 종류: 건조
비율: 24.50%, 다른 종류 평균 비율: 1.12%, 비율 차이: 23.37%

기상상태에서 '중' 또는 '상' 비율이 가장 높은 종류: 맑음
비율: 24.29%, 다른 종류 평균 비율: 0.94%, 비율 차이: 23.35%

도로형태에서 '중' 또는 '상' 비율이 가장 높은 종류: 기타
비율: 25.76%, 다른 종류 평균 비율: 0.54%, 비율 차이: 25.22%

가해운전자 차종에서 '중' 또는 '상' 비율이 가장 높은 종류: 승용
비율: 15.53%, 다른 종류 평균 비율: 3.37%, 비율 차이: 12.16%

가해운전자 성별에서 '중' 또는 '

In [31]:
# 위험도가 '중' 또는 '상'인 데이터를 필터링
df_filtered = df[df['위험도'].isin(['중', '상'])]

# 결과를 저장할 딕셔너리
result = {}

# 각 독립변수에 대해 반복
independent_vars = ['시간', '계절', '요일', '주말여부', '지역', '사고내용', '사고유형', '법규위반',
                    '노면상태', '기상상태', '도로형태', '가해운전자 차종', '가해운전자 성별',
                    '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 상해정도', '시간대']

for var in independent_vars:
    # 각 독립변수의 전체 데이터 수를 계산
    total_counts = df[var].value_counts()

    # '중' 또는 '상'인 데이터의 개수를 구하기
    filtered_counts = df_filtered[var].value_counts()

    # '중' + '상' 위험도의 비율을 계산
    proportions = (filtered_counts / len(df)).fillna(0)

    # 가장 비율이 높은 종류를 찾기
    max_category = proportions.idxmax()
    max_proportion = proportions.max()

    # 다른 종류들의 평균 비율을 계산
    other_mean_proportion = proportions.drop(max_category).mean()

    # 가장 높은 비율과 다른 종류들의 평균 비율 대비 얼마나 높은지 계산
    relative_increase = (max_proportion - other_mean_proportion)

    # 결과 저장
    result[var] = {
        "'중' 또는 '상' 비율이 가장 높은 종류": max_category ,
        "비율": round(max_proportion,4)*100,
        "다른 종류 평균 비율": round(other_mean_proportion, 4)*100,
        "비율 차이": round(relative_increase  ,4)*100
    }

# 딕셔너리를 데이터프레임으로 변환
df_result = pd.DataFrame(result)

# 결과 출력
display(df_result)


,시간,계절,요일,주말여부,지역,사고내용,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 상해정도,시간대
'중' 또는 '상' 비율이 가장 높은 종류,14.00,가을,금,주중,경기도,중상사고,차대차,안전운전불이행,건조,맑음,기타,승용,남,상해없음,승용,남,중상,주간
비율,1.83,11.97,4.59,21.04,9.5,24.32,26.37,21.15,24.5,24.29,25.76,15.53,25.67,19.25,16.79,21.98,14.7,18.87
다른 종류 평균 비율,1.18,5.68,4.07,7.95,1.39,4.67,1.31,1.31,1.12,0.94,0.54,3.37,3.33,1.95,1.22,1.75,2.04,10.12
비율 차이,0.65,6.29,0.53,13.09,8.11,19.65,25.06,19.85,23.37,23.35,25.22,12.16,22.34,17.3,15.57,20.23,12.66,8.75


# 종속변수의 정규성 검정
 - 카이제곱 검정

# 독립변수와 종속변수의 유의성 검정

In [ ]:
# 독립변수
X  = df.select_dtypes(include=['object']).columns
X = [col for col in X if col != '위험도']

# 종속 변수
y = df['위험도']

# 유의미한 변수 저장 리스트
significant_vars = []

# 각 독립 변수에 대해 카이제곱 검정 수행
for x in X:
    # 교차표 생성
    contingency_table = pd.crosstab(df[x], y)

    # 카이제곱 검정 수행
    chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)

    # p-value가 유의수준보다 작으면 리스트에 추가
    alpha = 0.05
    print(f"var : {x}, P-value : {p_value}")
    if p_value < alpha:
        significant_vars.append(x)


# 결과 출력
print("유의미한 관계가 있는 변수들:")
print(significant_vars)

var : 시간, P-value : 0.018851732840198945
var : 계절, P-value : 0.5777325209283746
var : 요일, P-value : 0.07670631340518676
var : 주말여부, P-value : 0.9364602777363095
var : 지역, P-value : 8.334562775696184e-13
var : 사고내용, P-value : 0.0
var : 사고유형, P-value : 0.06966222639708664
var : 법규위반, P-value : 4.2153149883931904e-10
var : 노면상태, P-value : 0.01720778785058968
var : 기상상태, P-value : 0.016306696757429847
var : 도로형태, P-value : 0.3088670921135362
var : 가해운전자 차종, P-value : 3.4923400728829543e-07
var : 가해운전자 성별, P-value : 0.04220600325607778
var : 가해운전자 상해정도, P-value : 3.6110379746577025e-104
var : 피해운전자 차종, P-value : 1.0806358791405787e-05
var : 피해운전자 성별, P-value : 0.0053492807163085885
var : 피해운전자 상해정도, P-value : 5.537696813323597e-183
var : 시간대, P-value : 0.3295270960924368
유의미한 관계가 있는 변수들:
['시간', '지역', '사고내용', '법규위반', '노면상태', '기상상태', '가해운전자 차종', '가해운전자 성별', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 상해정도']


# 라벨링 및 원핫인코딩

In [ ]:
# 원핫인코딩

X_col = significant_vars

df_one = df[X_col+['위험도']]
df_one = pd.get_dummies(df_one, columns=X_col)

# 타겟변수(종속변수) 라벨링
le = LabelEncoder()
df_one['위험도'] = le.fit_transform(df_one['위험도'])
df_one.head()

,위험도,시간_0,시간_1,시간_2,시간_3,시간_4,시간_5,시간_6,시간_7,시간_8,...,피해운전자 성별_없음,피해운전자 성별_여,피해운전자 상해정도_경상,피해운전자 상해정도_기타,피해운전자 상해정도_기타불명,피해운전자 상해정도_부상신고,피해운전자 상해정도_사망,피해운전자 상해정도_상해없음,피해운전자 상해정도_없,피해운전자 상해정도_중상
0,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,1,True,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,1,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
4,0,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


# 모델 학습

In [ ]:
# 데이터 나누기

y = df_one['위험도']
X = df_one.drop(columns=['위험도'])

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=123)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
# print(X_train.head())

(5760, 101)
(641, 101)
(5760,)
(641,)


## 분류모델
 - 종속변수 : '위험도'
 - 독립변수 : '시간', '지역', '사고내용', '법규위반', '노면상태', '기상상태', '가해운전자 차종', '가해운전자 성별', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 상해정도'


In [ ]:
# average (f1 score 변수)별 성능 테스트
average = ['micro','macro','weighted']

for i in average :
  print('--------------------------'+'average : '+i+'--------------------------')
  # AdaBoost 모델 생성 및 학습
  ada_model = AdaBoostClassifier(n_estimators=50, random_state=42)
  ada_model.fit(X_train, y_train)

  y_pred = ada_model.predict(X_test)
  print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred))
  print("AdaBoost f1score:", f1_score(y_test, y_pred, average=i))

  # XGBoost 모델 생성 및 학습
  xgb_model = xgb.XGBClassifier(n_estimators=50, use_label_encoder=False, eval_metric='mlogloss')
  xgb_model.fit(X_train, y_train)

  y_pred = xgb_model.predict(X_test)
  print("XGBoost Accuracy:", accuracy_score(y_test, y_pred))
  print("XGBoost f1score:", f1_score(y_test, y_pred, average=i))

  # Random Forest 모델 생성 및 학습
  rf_model = RandomForestClassifier()
  rf_model.fit(X_train, y_train)

  y_pred = rf_model.predict(X_test)
  print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
  print("Random Forest f1score:", f1_score(y_test, y_pred, average=i))

  # SVM 모델 생성 및 학습
  svm_model = SVC(kernel='linear')  # 커널 종류를 바꿔 다른 SVM 종류로 변경 가능
  svm_model.fit(X_train, y_train)

  y_pred = svm_model.predict(X_test)
  print("SVM Accuracy:", accuracy_score(y_test, y_pred))
  print("SVM f1score:", f1_score(y_test, y_pred, average=i))

--------------------------average : micro--------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Accuracy: 0.9188767550702028
AdaBoost f1score: 0.9188767550702028


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:35:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.9282371294851794
XGBoost f1score: 0.9282371294851794
Random Forest Accuracy: 0.9297971918876755
Random Forest f1score: 0.9297971918876755
SVM Accuracy: 0.9204368174726989
SVM f1score: 0.9204368174726989
--------------------------average : macro--------------------------


...

XGBoost Accuracy: 0.9282371294851794
XGBoost f1score: 0.6701494540898514
Random Forest Accuracy: 0.9282371294851794
Random Forest f1score: 0.6701494540898514
SVM Accuracy: 0.9204368174726989
SVM f1score: 0.5318152524167561
--------------------------average : weighted--------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Accuracy: 0.9188767550702028
AdaBoost f1score: 0.9031104339434746


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:36:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.9282371294851794
XGBoost f1score: 0.9182231456074065
Random Forest Accuracy: 0.9266770670826833
Random Forest f1score: 0.9158236413415882
SVM Accuracy: 0.9204368174726989
SVM f1score: 0.8944843499432781


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# n_estimators / learning_rate

# AdaBoost 모델 생성 및 학습
n_estimators = [10,50,100,150,200,1000]
learning_rate = [1,0.1,0.2,0.3]
for i in n_estimators:
  print(f'----------------------------n_estimators={i}-------------------------------')
  ada_model = AdaBoostClassifier(n_estimators=i
                                , random_state=1234)
  ada_model.fit(X_train, y_train)

  y_pred = ada_model.predict(X_test)
  print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred))
  print("AdaBoost f1score:", f1_score(y_test, y_pred, average='weighted'))

for j in learning_rate:
  print(f'----------------------------n_estimator = 50, learning_rate={j}-------------------------------')
  ada_model = AdaBoostClassifier(n_estimators=50
                              , learning_rate=j
                              , random_state=1234)
  ada_model.fit(X_train, y_train)

  y_pred = ada_model.predict(X_test)
  print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred))
  print("AdaBoost f1score:", f1_score(y_test, y_pred, average='weighted'))

  # n_estimator = 50, learning_rate=0.1 이 적당해보임

----------------------------n_estimators=10-------------------------------
AdaBoost Accuracy: 0.9173166926677067
AdaBoost f1score: 0.887879793475802
----------------------------n_estimators=50-------------------------------
AdaBoost Accuracy: 0.9188767550702028
AdaBoost f1score: 0.9031104339434746
----------------------------n_estimators=100-------------------------------
AdaBoost Accuracy: 0.9173166926677067
AdaBoost f1score: 0.9036132334210686
----------------------------n_estimators=150-------------------------------
AdaBoost Accuracy: 0.9157566302652106
AdaBoost f1score: 0.9040010839739121
----------------------------n_estimators=200-------------------------------
AdaBoost Accuracy: 0.9157566302652106
AdaBoost f1score: 0.9040010839739121
----------------------------n_estimators=1000-------------------------------
AdaBoost Accuracy: 0.9157566302652106
AdaBoost f1score: 0.9040010839739121
----------------------------n_estimator = 50, learning_rate=1-------------------------------
Ada

In [ ]:

warnings.filterwarnings("ignore", category=FutureWarning)

# [XGBoost] /n_estimators /learning_rate /learning_rate

n_estimators = [10,20,30,40,50]
learning_rate = [1,0.1,0.2,0.3]
max_depth = [3,4,5,6]


# XGBoost 모델 생성 및 학습

for i in n_estimators:
  print(f'----------------------------n_estimators={i}-------------------------------')
  xgb_model = xgb.XGBClassifier(n_estimators=i
                              , eval_metric='mlogloss'
                              ,random_state=1234)
  xgb_model.fit(X_train, y_train)

  y_pred = xgb_model.predict(X_test)
  print("XGBoost Accuracy:", accuracy_score(y_test, y_pred))
  print("XGBoost f1score:", f1_score(y_test, y_pred, average='weighted'))

print('=========================================================================================')

for j in learning_rate:
  print(f'----------------------------learning_rate={j}-------------------------------')
  xgb_model = xgb.XGBClassifier(n_estimators=50
                              ,learning_rate=j
                              , eval_metric='mlogloss'
                              ,random_state=1234)
  xgb_model.fit(X_train, y_train)

  y_pred = xgb_model.predict(X_test)
  print("XGBoost Accuracy:", accuracy_score(y_test, y_pred))
  print("XGBoost f1score:", f1_score(y_test, y_pred, average='weighted'))

print('=========================================================================================')

for k in max_depth:
  print(f'----------------------------max_depth={k}-------------------------------')
  xgb_model = xgb.XGBClassifier(n_estimators=50
                              ,learning_rate=1
                              ,max_depth=k, eval_metric='mlogloss',)
  xgb_model.fit(X_train, y_train)

  y_pred = xgb_model.predict(X_test)
  print("XGBoost Accuracy:", accuracy_score(y_test, y_pred))
  print("XGBoost f1score:", f1_score(y_test, y_pred, average='weighted'))
  # n_estimators=40 , learning_rate=0.1, max_depth=3 이 제일 좋아보임


----------------------------n_estimators=10-------------------------------
XGBoost Accuracy: 0.9391575663026521
XGBoost f1score: 0.9225747948031264
----------------------------n_estimators=20-------------------------------
XGBoost Accuracy: 0.9391575663026521
XGBoost f1score: 0.9240461872078541
----------------------------n_estimators=30-------------------------------
XGBoost Accuracy: 0.9391575663026521
XGBoost f1score: 0.9240461872078541
----------------------------n_estimators=40-------------------------------
XGBoost Accuracy: 0.9375975039001561
XGBoost f1score: 0.9254695645719035
----------------------------n_estimators=50-------------------------------
XGBoost Accuracy: 0.9282371294851794
XGBoost f1score: 0.9182231456074065
----------------------------learning_rate=1-------------------------------
XGBoost Accuracy: 0.9032761310452418
XGBoost f1score: 0.9050469470555471
----------------------------learning_rate=0.1-------------------------------
XGBoost Accuracy: 0.937597503900156